In [ ]:
import pandas            as pd
import plotly.express    as px
import matplotlib.pyplot as plt
import numpy             as np
import sklearn           as sk
import seaborn           as sns
import nltk
from nltk import word_tokenize
from nltk import NaiveBayesClassifier
from nltk.corpus    import stopwords
from nltk.classify  import accuracy 
from sklearn.tree                    import DecisionTreeClassifier
from sklearn.metrics                 import accuracy_score
from sklearn.datasets                import fetch_openml
from sklearn.ensemble                import RandomForestClassifier
from sklearn.neighbors               import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes             import MultinomialNB
from sklearn.neural_network          import MLPClassifier
from sklearn.model_selection         import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Giovanni Janzante Gianini

### RM 95306
### 2TDCR


#### Necessidade 2

In [ ]:
df = pd.read_csv('train.csv', encoding='UTF-8')

In [ ]:
df = df.dropna()

In [ ]:
df.sample(5)

In [ ]:
df

Criando uma coluna de numeros dependendo da linguagem

In [ ]:
def linguagem(linha):

    if linha['language']=='English':
        return 1
    elif linha['language']=='Spanish':
        return 2
    elif linha['language']=='Portuguese':
        return 3
    elif linha['language']=='Italian':
        return 4
    elif linha['language']=='French':
        return 5
    elif linha['language']=='Chinese':
        return 6
    else:
        return 7

def tamanho_mensagem(linha):
    return len(linha['text'])

df['language(num)'] = df.apply(linguagem, axis=1)


Removendo STOPWORDS de todos os textos em todas as linguas

In [ ]:

nltk.download('stopwords')

def processa_mensagem(linha):
    msg = linha["text"]
    msg = msg.lower()
    stops = set(stopwords.words("english") + stopwords.words("portuguese") + stopwords.words("spanish") + stopwords.words("chinese") + stopwords.words("italian") + stopwords.words("french"))
    
    # Vamos separar só as palavras que não são stopwords (e jogar fora o resto)
    palavras_filtradas = []
    for palavra in msg.split(' '):
        if palavra not in stops: # a palavra não é uma stopword
            palavras_filtradas.append(palavra)
            
    msg_sem_stopwords = ' '
    for palavra in palavras_filtradas:
        msg_sem_stopwords = msg_sem_stopwords + ' ' + palavra
    
    msg_sem_stopwords = " ".join([palavra for palavra in msg.split(" ")
                                  if palavra not in stops])
    
    return msg_sem_stopwords

df["text"] = df.apply(processa_mensagem, axis=1)
df

Criando variaveis e treinando o modelo para predizer.

In [ ]:
X ,y = df['text'] ,df['language(num)']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

vect = CountVectorizer() # não usamos os dados de teste nele!
vect.fit(X_train)

print(vect.get_feature_names_out()[1500:1510])
X_train = vect.transform(X_train)
X_test  = vect.transform(X_test)

Definindo o metodo para predizer o modelo de linguagem

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

Calculando a Acurácia de teste

In [ ]:
X_pred = clf.predict(X_test)
acc = accuracy_score(X_pred, y_test)
print(f"Acurácia: {round(acc, 4)*100}%")

Calculando a Acurácia do modelo de treinamento

In [ ]:
X_pred_train = clf.predict(X_train)
acc_train = accuracy_score(X_pred_train, y_train)
print(f"Acurácia: {round(acc_train, 4)*100}%")

### Realizar teste de frases com o nosso modelo

In [ ]:
mensagem_teste = "aujourd'hui je me suis réveillé le matin"
resposta = clf.predict(vect.transform([mensagem_teste]))
resposta = resposta[0]

if resposta == 1:
     res = "English"
elif resposta == 2:
     res = "Spanish"
elif resposta == 3:
     res = "Portuguese"
elif resposta == 4:
     res = "Italian"
elif resposta == 5:
     res = "French"
elif resposta == 6:
     res = "Chinese"
else:
     print("Erro")
print(res)

